## Galaxy Zoo All Questions

### Python files required to run ths notebook: data_loader.py, separate_training_validation.py

In [23]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import matplotlib.pyplot as plt
import numpy as np
import os

In [24]:
# Training settings
parser = argparse.ArgumentParser(description='Galaxy zoo project')
parser.add_argument('--data', type=str, default='data', metavar='D',
                    help="folder where data is located")
parser.add_argument('--crop_size', type=str, default=256, metavar='D',
                    help="Crop Size of images")
parser.add_argument('--resolution', type=str, default=64, metavar='D',
                    help="Final Resolution of images")
parser.add_argument('--batch_size', type=int, default=64, metavar='N',
                    help='input batch size for training (default: 64)')
parser.add_argument('--epochs', type=int, default=5, metavar='N',
                    help='number of epochs to train (default: 15)')
parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
                    help='learning rate (default: 0.01)')
parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
                    help='SGD momentum (default: 0.5)')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')

parser.add_argument('--question', type=int, default=0, metavar='N',
                    help='Question number for which training has to be done. If 0 then all questions')

parser.add_argument('--model_directory', type=str, default="models", metavar='N',
                    help='directory to store models')

parser.add_argument('--validation_length', type=int, default=20, metavar='N',
                    help='length of valiudation set')

parser.add_argument('--train_length', type=int, default=1000, metavar='N',
                    help='length of train set')

input_args = ""
args = parser.parse_args(input_args)
torch.manual_seed(args.seed)

In [25]:
### Data Initialization and Loading
from data_loader import initialize_data, loader
initialize_data(args.data) 

In [26]:
from YLabelCreate import getYlabel

label_ids_training, label_ids_validation, label_values_training, label_values_validation = getYlabel(args.train_length,args.validation_length)


In [27]:
crop_size = args.crop_size
resolution = args.resolution
batch_size = args.batch_size

questions = args.question
shuffle=True

transformations = transforms.Compose([
    transforms.Scale(256),
    transforms.CenterCrop(224),
    transforms.ToTensor()])


/Users/amaladeshpande/anaconda3/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg/torchvision/transforms/transforms.py:188: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


In [28]:
train_loader = loader(label_ids_training, label_values_training, crop_size, resolution, batch_size, shuffle, questions)
shuffle=False
val_loader=loader(label_ids_validation, label_values_validation, crop_size, resolution, batch_size, shuffle, questions)

In [29]:
from Model_All_Questions import Net

In [30]:
model = Net()
optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)
loss_train=nn.MSELoss()
loss_val=nn.MSELoss(reduction='sum')

In [31]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target).float()
        optimizer.zero_grad()
        output = model(data).float()
        loss = loss_train(output, target)
        loss = Variable(loss, requires_grad = True)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                                                                           epoch, batch_idx * len(data), len(train_loader.dataset),
                                                                           100. * batch_idx / len(train_loader), loss.data[0]))



In [32]:
def validation():
    model.eval()
    validation_loss = 0
    correct = 0
    for data, target in val_loader:
        data, target = Variable(data, volatile=True), Variable(target).float()
        output = model(data).float()
        loss=loss_val(output, target)
        loss = Variable(loss, requires_grad = True)
        validation_loss += loss # sum up batch loss
        
    validation_loss /= len(val_loader.dataset)
    


    print('\nValidation set: Average loss: {:.4f}\n'.format(validation_loss, correct))
        
            

In [33]:

if not os.path.exists(args.model_directory):
    os.makedirs(args.model_directory)
    
for epoch in range(1, args.epochs + 1):
    train(epoch)
   
    validation()
    if(epoch%10==0):
        model_file = args.model_directory+'/model_' + str(epoch) + '.pth'
        torch.save(model.state_dict(), model_file)

/Users/amaladeshpande/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/Users/amaladeshpande/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  


Train Epoch: 1 [0/1000 (0%)]	Loss: 0.041357
Train Epoch: 1 [640/1000 (62%)]	Loss: 0.049341


/Users/amaladeshpande/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  



Validation set: Average loss: 1.7185

Train Epoch: 2 [0/1000 (0%)]	Loss: 0.043578
Train Epoch: 2 [640/1000 (62%)]	Loss: 0.048924

Validation set: Average loss: 1.7185

Train Epoch: 3 [0/1000 (0%)]	Loss: 0.043270
Train Epoch: 3 [640/1000 (62%)]	Loss: 0.043562

Validation set: Average loss: 1.7185

Train Epoch: 4 [0/1000 (0%)]	Loss: 0.041408
Train Epoch: 4 [640/1000 (62%)]	Loss: 0.038662

Validation set: Average loss: 1.7185

Train Epoch: 5 [0/1000 (0%)]	Loss: 0.047546
Train Epoch: 5 [640/1000 (62%)]	Loss: 0.043302

Validation set: Average loss: 1.7185

